# Datenbanksetup

Das Notebook dient dazu, das Schema für die GraphDB zu erstellen und die Spielwelt zu generieren.

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv(dotenv_path='../.env')

driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

# Queryhelper
def run_queries(queries, params=None):
    
    # Ergebnisliste
    results = []

    # Liste durchgehen. Muss immer eine Liste sein, keine Normalisierung aktuell.
    for query in queries:

        # Session öffnen
        with driver.session() as session:

            # Query
            result = session.run(query, params or {})

            # Ergebnisse wegspeichern
            records = [r.data() for r in result]
            results.extend(records)

    return results

Docker starten ;-)
URI: bolt://localhost:7687
UI:  http://localhost:7474


In [ ]:
# Constrains festlegen
constrain_queries = [
    'CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE',
    'CREATE CONSTRAINT item_id     IF NOT EXISTS FOR (i:Item)     REQUIRE i.id IS UNIQUE',
    'CREATE CONSTRAINT npc_id      IF NOT EXISTS FOR (n:NPC)      REQUIRE n.id IS UNIQUE',
    'CREATE CONSTRAINT player_id   IF NOT EXISTS FOR (p:Player)   REQUIRE p.id IS UNIQUE',
]

run_queries(constrain_queries)
print()

# Index festlegen
entity_indexes = [
    "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
    "CREATE INDEX item_name     IF NOT EXISTS FOR (i:Item)     ON (i.name)"
]

run_queries(entity_indexes)

In [ ]:
player_knot = [
    """
    MERGE (p:Player {id: 'player_no1'})
    ON CREATE SET p.name = "Player", p.description = "Hier könnte Dein Name stehen!"
    RETURN p
    """ 
]

run_queries(player_knot)

In [ ]:
# Orte/Räume: id, name, beschreibung
location_knots = [
    """
    MERGE (l:Location {id: 'taverne'})
    ON CREATE SET 
        l.name = "Mo's Taverne", 
        l.description = "Eine alte Taverne, etwas heruntergekommen aber ein Kultobjekt."
    RETURN l
    """,
    """
    MERGE (l:Location {id: 'marktplatz'})
    ON CREATE SET 
        l.name = "Marktplatz", 
        l.description = "Das ist der Markplatz des kleinen, beschaulichen Ortes."
    RETURN l
    """,
    """
    MERGE (l:Location {id: 'finsterwald'})
    ON CREATE SET 
        l.name = "Finsterwald", 
        l.description = "Ein dunkler, übelriechender Wald voller Geister und anderen Ängsten."
    RETURN l
    """
]

run_queries(location_knots)

In [ ]:
# Items: id, name, beschreibung
item_knots = [
    """
    MERGE (i:Item {id: 'schluessel'})
    SET
        i.name = 'Schlüssel',
        i.description = 'Ein verzauberter Schlüssel der nur in ein ganz bestimmtes Schloss passt.'
    """,
    """
    MERGE (i:Item {id: 'truhe'})
    SET
        i.name = 'Truhe',
        i.description = 'Eine alte Truhe mit einem rostigen Schloss.'
    """
]

run_queries(item_knots)

In [ ]:
# NPCs: id, name, beschreibung
npc_knots = [
    """
    MERGE (n:NPC{id: 'wirt'})
    SET
        n.name = 'Schenk',
        n.description = 'Ein alter, grummiger Wirt, der seinen Gästen stets zu wenig einschenkt.'
    """
]

run_queries(npc_knots)

In [ ]:
# Persons start
player_edges = [
    """
    MATCH (start:Player {id: 'player_no1'}), (ende:Location {id: 'marktplatz'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """,
    """
    MATCH (start:NPC {id: 'wirt'}), (ende:Location {id: 'taverne'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """
]

run_queries(player_edges)

In [ ]:
# Wege
location_edges = [
    """
    MATCH 
        (m:Location {id: 'marktplatz'}),
        (t:Location {id: 'taverne'}),
        (f:Location {id: 'finsterwald'})
    MERGE (m)-[:ERREICHT]->(t)
    MERGE (t)-[:ERREICHT]->(m)
    MERGE (m)-[:ERREICHT]->(f)
    MERGE (f)-[:ERREICHT]->(m)
    """
]

run_queries(location_edges)

In [ ]:
# Items
item_edges = [
    """
    MATCH 
        (ta:Location {id: 'taverne'}), 
        (f:Location {id: 'finsterwald'}),
        (s:Item {id: 'schluessel'}),
        (tr:Item {id: 'truhe'})
    MERGE (s)-[:IST_IN]->(ta)
    MERGE (tr)-[:IST_IN]->(f)
    """
]

run_queries(item_edges)

In [8]:
# Quest
quest_edge = [
    """
    MATCH
        (s:Item {id: 'schluessel'}), 
        (t:Item {id: 'truhe'})
    MERGE (s)-[:OEFFNET]->(t)
    """
]

run_queries(quest_edge)

[]